In [259]:
# -*- coding: utf-8 -*-
%matplotlib inline

import pandas as pd
import numpy as np
import math
import matplotlib as mplstyle
import matplotlib.pyplot as plt
import locale
import arrow
from locale import atof
from dateutil import parser
from datetime import datetime

### Functions

In [260]:
# Remove punctuation
def removeFormat(item):
    try:
        tempVal = item.replace(',','')
        val = int(tempVal)
        return val
    except:
        return float('nan')

### Imports

In [261]:
df_vacc = pd.read_csv('../data/imports/dtp1.csv')
df_pop = pd.read_csv('../data/imports/population-under14.csv')
df_gdp = pd.read_csv('../data/imports/GDP2016.csv')
df_ppp = pd.read_csv('../data/imports/PPP.csv')
df_pppMeta = pd.read_csv('../data/imports/PPP_meta.csv')
df_graphicExample = pd.read_csv('../data/imports/graphic-copy-sample.csv')
df_completeCountryList = pd.read_csv('../data/exports/complete-country-list.csv')

### Analysis

In [262]:
# Merge vaccine data and GDP
df_vacGdpMerge = df_vacc.merge(df_gdp, on='iso3', how='outer')

In [263]:
# Prep population for merge 
df_pop = df_pop.rename(columns = {'Region, country or area' : 'country'})

# Merge vacGdp and population
df_vacGdpPopMerge = df_vacGdpMerge.merge(df_pop, on='country', how='outer')

In [264]:
# Clean merge
df_vacGdpPopMergeClean = df_vacGdpPopMerge.drop(['vaccine', 'economy', ' Male ', ' Female '], axis=1)
df_vacGdpPopMergeClean = df_vacGdpPopMergeClean.rename(columns = {'millions of US dollars':'GDP',' 0-14 (percentage) ':'Pop % under 14','Number under age 14':'Pop # under 14'})
df_vacGdpPopMergeClean['Pop # under 14'] = df_vacGdpPopMergeClean['Pop # under 14'].apply(removeFormat)
df_vacGdpPopMergeClean

,unicef_region,iso3,country,2016,2015,2014,2013,2012,2011,2010,...,1982,1981,1980,GDP,Total (in thousdands),Pop % under 14,15-24,25-59,60+,Pop # under 14
0,South Asia,AFG,Afghanistan,73.0,73.0,73.0,76.0,78.0,78.0,72.0,...,24.0,NaN,22.0,"19,469","35,530",43.0,21.0,31,4.0,15277900.0
1,Eastern Europe & Central Asia,ALB,Albania,99.0,99.0,99.0,99.0,99.0,99.0,99.0,...,98.0,97.0,97.0,"11,927","2,930",17.0,16.0,48,19.0,498100.0
2,Middle East & North Africa,DZA,Algeria,96.0,99.0,99.0,99.0,99.0,99.0,99.0,...,NaN,NaN,NaN,"156,080","41,318",29.0,15.0,zz,9.0,11982220.0
3,Western Europe,AND,Andorra,99.0,99.0,99.0,98.0,99.0,99.0,99.0,...,NaN,NaN,NaN,..,77,NaN,NaN,NaN,NaN,NaN
4,Eastern & Southern Africa,AGO,Angola,79.0,77.0,81.0,79.0,83.0,88.0,93.0,...,NaN,NaN,NaN,"89,633","29,784",47.0,19.0,30,4.0,13998480.0
5,Latin America & Caribbean,ATG,Antigua and Barbuda,99.0,99.0,99.0,99.0,99.0,99.0,99.0,...,92.0,92.0,77.0,"1,449",102,24.0,17.0,49,11.0,24480.0
6,Latin America & Caribbean,ARG,Argentina,97.0,94.0,98.0,94.0,94.0,94.0,95.0,...,80.0,73.0,69.0,"545,866","44,271",25.0,16.0,44,15.0,11067750.0
7,Eastern Europe & Central Asia,ARM,Armenia,97.0,97.0,97.0,97.0,98.0,98.0,98.0,...,NaN,NaN,NaN,"10,547","2,930",20.0,13.0,50,17.0,586000.0
8,East Asia & Pacific,AUS,Australia,98.0,97.0,92.0,92.0,92.0,92.0,97.0,...,72.0,65.0,58.0,"1,204,616","24,451",19.0,13.0,47,21.0,4645690.0
9,Western Europe,AUT,Austria,99.0,99.0,99.0,99.0,99.0,99.0,98.0,...,96.0,96.0,96.0,"386,428","8,735",14.0,11.0,50,25.0,1222900.0


In [265]:
# Columns for condesnced df
cols = ['unicef_region','iso3','country','GDP','2016','1996','1986','Pop % under 14','Pop # under 14']

# Apply columns
df_vacGdpPopMergeCond = df_vacGdpPopMergeClean[cols]
df_vacGdpPopMergeCond = df_vacGdpPopMergeCond[np.isfinite(df_vacGdpPopMergeCond['2016'])]
df_vacGdpPopMergeCond.sort_values('GDP')
df_vacGdpPopMergeCond.to_csv('../data/exports/df_vacGdpPopMergeCond.csv')

In [266]:
# Prep to merge PPP and PPP Meta

# Raname columns for merge
df_ppp = df_ppp.rename(columns = {'Country Code':'iso3','2016':'PPP 2016'})
df_pppMeta = df_pppMeta.rename(columns = {'Country Code':'iso3'})

# Define columns to keep
pppCols = ['iso3','PPP 2016']
pppMetaCols = ['iso3','IncomeGroup']

# Apply columns to dataframes
df_pppCond = df_ppp[pppCols]
df_pppMetaCond = df_pppMeta[pppMetaCols]

# Merge PPP files together
df_pppMerge = df_pppCond.merge(df_pppMetaCond,on='iso3',how='outer')

In [267]:
df_pppMerge

,iso3,PPP 2016,IncomeGroup
0,ABW,NaN,High income
1,AFG,1876.544682,Low income
2,AGO,6499.067104,Lower middle income
3,ALB,11928.536560,Upper middle income
4,AND,NaN,High income
5,ARB,16697.596840,NaN
6,ARE,72418.610860,High income
7,ARG,19934.372490,Upper middle income
8,ARM,8817.953996,Lower middle income
9,ASM,NaN,Upper middle income


In [268]:
# Merge df_vacGdpPopMergeCond and df_pppMerge then rename columns
df_finalMerge = df_vacGdpPopMergeCond.merge(df_pppMerge, on='iso3', how='outer')
df_finalMergeClean = df_finalMerge.rename(columns = {'GDP':'GDP 2016','2016':'DTP1-16','1996':'DTP1-96','1986':'DTP1-86'})
# list(df_finalMerge)

# Filter for blanks
df_finalMergeClean = df_finalMergeClean[np.isfinite(df_finalMerge['2016'])]

# Reorder
list(df_finalMergeClean)
cols = ['unicef_region','iso3','country','IncomeGroup','GDP 2016','PPP 2016','Pop % under 14','Pop # under 14','DTP1-16','DTP1-96','DTP1-86']
df_finalMergeClean = df_finalMergeClean[cols]

df_finalMergeClean.to_csv('../data/exports/df_finalMergeClean.csv')

In [269]:
df_finalMergeClean.sort_values('country')

,unicef_region,iso3,country,IncomeGroup,GDP 2016,PPP 2016,Pop % under 14,Pop # under 14,DTP1-16,DTP1-96,DTP1-86
0,South Asia,AFG,Afghanistan,Low income,"19,469",1876.544682,43.0,15277900.0,73.0,56.0,32.0
1,Eastern Europe & Central Asia,ALB,Albania,Upper middle income,"11,927",11928.536560,17.0,498100.0,99.0,98.0,98.0
2,Middle East & North Africa,DZA,Algeria,Upper middle income,"156,080",15074.924810,29.0,11982220.0,96.0,96.0,92.0
3,Western Europe,AND,Andorra,High income,..,NaN,NaN,NaN,99.0,NaN,NaN
4,Eastern & Southern Africa,AGO,Angola,Lower middle income,"89,633",6499.067104,47.0,13998480.0,79.0,53.0,31.0
5,Latin America & Caribbean,ATG,Antigua and Barbuda,High income,"1,449",22413.483800,24.0,24480.0,99.0,99.0,98.0
6,Latin America & Caribbean,ARG,Argentina,Upper middle income,"545,866",19934.372490,25.0,11067750.0,97.0,94.0,86.0
7,Eastern Europe & Central Asia,ARM,Armenia,Lower middle income,"10,547",8817.953996,20.0,586000.0,97.0,95.0,NaN
8,East Asia & Pacific,AUS,Australia,High income,"1,204,616",46789.927240,19.0,4645690.0,98.0,95.0,91.0
9,Western Europe,AUT,Austria,High income,"386,428",50077.832610,14.0,1222900.0,99.0,96.0,96.0


In [274]:
# Assign columns for export
cols = ['iso3','Pop # under 14','DTP1-16']

# Drop unnecessary columns
df_mapDetailsCond = df_finalMergeClean[cols]

# Rename 
df_mapDetailsCond = df_mapDetailsCond.rename(columns={'country':'name','Pop # under 14':'population'})
df_mapDetailsCond.to_csv('TEST.csv')


In [275]:
# Merge with graphic example then drop rows with empty DTP cells and unnecessary columns
df_completeCountryList = df_completeCountryList.rename(columns={'ADM0_A3_IS':'iso3','NAME':'name'})
df_graphicSpreadsheet = df_completeCountryList.merge(df_mapDetailsCond, on='iso3', how='outer').drop('Unnamed: 0',axis=1)
df_graphicExport = df_graphicSpreadsheet.merge(df_graphicExample,on='name',how='outer')

df_graphicExportCond = df_graphicSpreadsheet[np.isfinite(df_graphicSpreadsheet['DTP1-16'])]
# df_graphicExportFinal = df_graphicExport[np.isfinite(df_graphicExport['DTP1-16'])]
df_graphicExportCond

,iso3,ISO_A3,ISO_N3,ABBREV,GEOUNIT,name,NAME_LONG,population,DTP1-16
1,ATG,-99,-99,Ant.,Antigua,Antigua,Antigua,24480.0,99.0
2,ATG,-99,-99,Barb.,Barbuda,Barbuda,Barbuda,24480.0,99.0
3,AFG,AFG,4,Afg.,Afghanistan,Afghanistan,Afghanistan,15277900.0,73.0
4,AGO,AGO,24,Ang.,Angola,Angola,Angola,13998480.0,79.0
6,ALB,ALB,8,Alb.,Albania,Albania,Albania,498100.0,99.0
8,AND,AND,20,And.,Andorra,Andorra,Andorra,NaN,99.0
9,ARE,ARE,784,U.A.E.,United Arab Emirates,United Arab Emirates,United Arab Emirates,1316000.0,99.0
10,ARG,ARG,32,Arg.,Argentina,Argentina,Argentina,11067750.0,97.0
11,ARM,ARM,51,Arm.,Armenia,Armenia,Armenia,586000.0,97.0
14,AUS,-99,36,A.C.Is.,Ashmore and Cartier Islands,Ashmore and Cartier Is.,Ashmore and Cartier Islands,4645690.0,98.0


In [272]:
# Export
df_graphicExportFinal.to_csv('../data/exports/df_graphicExportFinal.csv')